In [1]:
from src.utils import parse_log, check_completion, view_log_summary
import pandas as pd
from datetime import datetime


In [ ]:
# Merge df_max_run with result_df to get the overall_completion status
merged_df = df_max_run.merge(result_df[['email', 'overall_completion']], left_on='account', right_on='email', how='left')

# Filter accounts where is_run_today is False or (is_run_today is True and overall_completion is False)
rerun_needed_accounts = merged_df[(merged_df['is_run_today'] == False) | 
                                  ((merged_df['is_run_today'] == True) & (merged_df['overall_completion'] == False))]['account'].tolist()

rerun_needed_accounts


Parsing log file: /Users/yaleipeng/Projects/MS-Rewards-Farmer/logs/activity.log


,account,run_index,ts_start,ts_end,overall_completion,task.daily_set,task.desktop_searches,task.mobile_searches,task.read_articles,is_run_today
0,yaleipeng@gmail.com,2,"2024-11-24 06:31:51,918","2024-11-24 06:46:38,182",True,True,True,True,True,True


In [19]:
df_max_run

,account,run_index,ts_start,ts_end,overall_completion,task.daily_set,task.desktop_searches,task.mobile_searches,task.read_articles,is_run_today
0,yaleipeng@gmail.com,2,"2024-11-24 06:31:51,918","2024-11-24 06:46:38,182",True,True,True,True,True,True


In [ ]:
log_summary = parse_log(
    # log_file_path=r"/Users/yaleipeng/Projects/MS-Rewards-Farmer/logs/activity.log.2024-11-23"
    )
# df = view_log_summary(summary=log_summary, agg_runs=True)
# display(df)
df_completion = check_completion(log_summary, ret_type="df", agg_runs=True)

df_max_run = df_completion.loc[df_completion.groupby('account')['run_index'].idxmax()].reset_index(drop=True)
today = datetime.today().date()
df_max_run['is_run_today'] = df_max_run['ts_start'].apply(pd.Timestamp).dt.date == today

# Filter accounts where is_run_today is False or (is_run_today is True and overall_completion is False)
rerun_needed_accounts = df_max_run[(df_max_run['is_run_today'] == False) | 
                                  ((df_max_run['is_run_today'] == True) & (df_max_run['overall_completion'] == False))]['account'].tolist()

rerun_needed_accounts


Parsing log file: /Users/yaleipeng/Projects/MS-Rewards-Farmer/logs/activity.log


[]

In [ ]:
check_completion(log_summary, ret_type="df")


In [ ]:
completion_dict = check_completion(log_summary)
# Get the last run dict for each account
last_run_dict = {account: runs[max(runs.keys())] for account, runs in completion_dict.items()}
last_run_dict

In [ ]:
# Assuming df has columns 'email', 'ts_start', and 'ts_end'
today = datetime.today().date()

# Group by email and calculate min ts_start, max ts_end, and the bool column
result_df = df.groupby('email').agg(
    min_ts_start=('ts_start', 'min'),
    max_ts_end=('ts_end', 'max')
).reset_index()

result_df['run_today'] = result_df['min_ts_start'].dt.date == today
# Extract overall completion flag
result_df['overall_completion'] = result_df['email'].map(lambda x: completion_dict.get(x, (False, {}))[0])

# Extract task completion flags
task_completion_flags = result_df['email'].map(lambda x: completion_dict.get(x, (False, {}))[1])
task_completion_df = pd.DataFrame(task_completion_flags.tolist(), index=result_df.index)

# Combine the task completion flags with the result_df
result_df = pd.concat([result_df, task_completion_df], axis=1)
result_df

In [ ]:
completion_dict